# Just generate example data

In [ ]:
import numpy as np
import scipy.ndimage

def generate_organic_phantom(filename='example.npz'):
    print("Generating organic asymmetric data...")
    
    # Dimensions
    D, H, W = 60, 512, 512
    
    # --- 1. Generate "Structures" (Low Frequency) ---
    # We generate a smaller random grid and zoom it up. This creates large asymmetric blobs.
    # Alternatively, we smooth a full size grid heavily.
    print("Generating base structures (large blobs)...")
    np.random.seed(42) # For reproducibility
    base_noise = np.random.normal(0, 1, (D, H, W)).astype(np.float32)
    
    # Sigma (smoothing) controls the "size" of the blobs. 
    # [3, 20, 20] means blobs are elongated in Z (like a body) and large in X/Y.
    structures = scipy.ndimage.gaussian_filter(base_noise, sigma=[4, 30, 30])
    
    # Normalize structures to 0..1 range roughly
    structures = (structures - structures.min()) / (structures.max() - structures.min())
    
    # --- 2. Create the Signal (HU Mapping) ---
    # Map the 0..1 blobs to HU range [-1000, 2000]
    # We apply a slight curve (power) to make edges sharper/more organic
    signal_curve = structures ** 1.5 
    clean_image = (signal_curve * 3000) - 1000  # Maps 0->-1000, 1->2000
    
    # --- 3. Generate Mask (Clean) ---
    # Threshold the structures BEFORE adding texture noise
    print("Creating mask from structures...")
    mask = np.where(clean_image > 130, 1, 0).astype(np.uint16)
    
    # --- 4. Add "Tissue Texture" (Mid Frequency Noise) ---
    # Instead of white noise (fine grain), we use "pink noise" (slightly smoothed)
    print("Adding tissue texture...")
    texture_noise = np.random.normal(0, 1, (D, H, W)).astype(np.float32)
    texture = scipy.ndimage.gaussian_filter(texture_noise, sigma=1.0) # Light smoothing
    
    # Scale texture amplitude (e.g., +/- 100 HU)
    # We multiply by clean_image normalized to make dense areas noisier (optional realism)
    noisy_image = clean_image + (texture * 100)
    
    # --- 5. Add "Asymmetric Artifact" (Optional Interest) ---
    # Let's add a large "dense" streak through the volume to break symmetry further
    # e.g., a diagonal gradient
    z, y, x = np.indices((D, H, W))
    gradient = (x + y + z*2) / (W + H + D)
    noisy_image += gradient * 200 # Slight drift in density
    
    # --- 6. Finalize ---
    final_image = np.clip(noisy_image, -32768, 32767).astype(np.int16)
    
    # Metadata
    origin = np.array([0, 0, 0])
    spacing = np.array([2.0, 0.7, 0.7]) 

    print(f"Saving to {filename}...")
    np.savez_compressed(
        filename,
        image=final_image,
        mask=mask,
        origin=origin,
        spacing=spacing
    )
    print("Done. Image range: [{}, {}]".format(final_image.min(), final_image.max()))

In [ ]:
generate_organic_phantom()

# Main code demonstrating slicers

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from dicom_utils import DicomWidget, DicomSlicer
from dicom_utils import InteractiveSlicer





data = np.load('example.npz')
mask = data['mask']
image = data['image']
origin = data['origin']
spacing = data['spacing']

## `InteractiveSlicer` : ipyevents mouse interactions

In [ ]:
slicer = DicomSlicer(
    image_array=image, 
    mask=mask)


app = InteractiveSlicer(
    dicom_slicer=slicer, 
    debug_overlay=True, 
    fps=20,             # Smoother interactions
    show_status=True    # Show the status bar
)

display(app.display())

In [ ]:
app.slicer.state['hu'] =(-100,1500) 



## `DicomWidget` no mouse interaction 

In [ ]:
w = DicomWidget(
    image_array=image, 
    mask=mask)


In [ ]:
w.display()